In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_diagonal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,1255.25,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00
3,-0.00,299.34,-0.00,0.00,0.00,0.00,0.00,-0.00
4,0.00,-0.00,63.06,-0.00,-0.00,0.00,-0.00,0.00
5,-0.00,0.00,-0.00,12.61,0.00,0.00,0.00,-0.00
6,-0.00,0.00,-0.00,0.00,2.51,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,0.53,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.16,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00003,-0.00004,-0.00002,-0.00005,0.00001
3,-0.00001,1.00000,-0.00000,0.00003,0.00001,0.00004,0.00000,-0.00003
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00006,-0.00002,0.00003
5,-0.00003,0.00003,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00004
6,-0.00004,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00002,0.00004,0.00006,0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00005,0.00000,-0.00002,0.00001,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.008614901679492502

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.97208863e+05 1.09731661e+04 4.73535385e+02 1.81995706e+01
 6.77877314e-01 2.76314431e-02 1.87784431e-03 4.56730346e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999324,-0.036038,-0.007118,-0.001423,-0.000284,-0.000060,-0.000018,-0.000005
1,0.035741,0.998652,-0.036943,-0.007015,-0.001394,-0.000295,-0.000087,-0.000027
2,0.008359,0.036328,0.998394,-0.041879,-0.007954,-0.001679,-0.000496,-0.000150
3,0.002000,0.008381,0.041095,0.997704,-0.051983,-0.010518,-0.003078,-0.000943
4,0.000495,0.002068,0.009840,0.050605,0.995687,-0.074062,-0.020520,-0.006232
5,0.000134,0.000560,0.002660,0.013340,0.069046,0.984402,-0.155673,-0.042030
6,0.000055,0.000228,0.001085,0.005419,0.027278,0.134410,0.943749,-0.300828
7,0.000036,0.000152,0.000721,0.003612,0.018119,0.085373,0.290987,0.952731


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006759939959474215,
 0.0013476198215669477,
 0.0016062292507983411,
 0.002295805848875676,
 0.004312935173527488,
 0.015598303893137211,
 0.05625140077298718,
 0.047269212495748225]